In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import sys
import json
from pathlib import Path
from datasets import load_dataset
from dotenv import load_dotenv

sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), '..'))

load_dotenv()

ds = load_dataset("SimpleStories/SimpleStories", split="train")
print("Dataset features:")
print(ds.features)
print(f"\nDataset size: {len(ds)}")
print(f"\nFirst example keys: {list(ds[0].keys())}")

Dataset features:
{'story': Value('string'), 'topic': Value('string'), 'theme': Value('string'), 'style': Value('string'), 'feature': Value('string'), 'grammar': Value('string'), 'persona': Value('string'), 'initial_word_type': Value('string'), 'initial_letter': Value('string'), 'word_count': Value('int64'), 'character_count': Value('int64'), 'num_paragraphs': Value('int64'), 'avg_word_length': Value('float64'), 'avg_sentence_length': Value('float64'), 'flesch_reading_ease': Value('float64'), 'flesch_kincaid_grade': Value('float64'), 'dale_chall_readability_score': Value('float64'), 'num_stories_in_completion': Value('int64'), 'expected_num_stories_in_completion': Value('int64'), 'generation_id': Value('string'), 'model': Value('string')}

Dataset size: 2115696

First example keys: ['story', 'topic', 'theme', 'style', 'feature', 'grammar', 'persona', 'initial_word_type', 'initial_letter', 'word_count', 'character_count', 'num_paragraphs', 'avg_word_length', 'avg_sentence_length', 'fles

## Step 1: Explore Dataset and Generate Leaf Nodes

First, we explore the dataset to propose 50 leaf nodes based on the dataset features.


In [17]:
# Run the dataset exploration script
from explore_dataset import explore_dataset, propose_leaf_nodes

# Explore the dataset
categorical_features, numeric_features = explore_dataset(sample_size=50000)

# Propose leaf nodes
leaf_nodes = propose_leaf_nodes(categorical_features, numeric_features)

print(f"\n✓ Generated {len(leaf_nodes)} leaf nodes")
print("\nFirst 10 leaf nodes:")
for i, node in enumerate(leaf_nodes[:10], 1):
    print(f"{i:2d}. {node['id']}: {node['description']}")


Analyzing 50000 examples from 2115696 total examples...

Most common categorical values:

topic (48 unique):
  hidden treasures: 1123
  magical lands: 1105
  bygone eras: 1094
  the arts: 1092
  cultural traditions: 1086
  seasonal changes: 1080
  giant creatures: 1079
  mystical creatures: 1076
  time travel: 1073
  lost civilizations: 1073

theme (63 unique):
  Magic: 878
  Deception: 853
  Helping Others: 852
  Agency: 847
  Innovation: 840
  Kindness: 831
  Problem-Solving: 827
  Humor: 819
  Growth: 817
  Hardship: 814

style (23 unique):
  minimalist: 2325
  classic: 2255
  lighthearted: 2216
  playful: 2215
  modern: 2209
  surreal: 2208
  philosophical: 2199
  humorous: 2181
  tragic: 2180
  fable-like: 2180

feature (26 unique):
  a flashback: 2021
  circular narrative structure: 2013
  a cliffhanger: 1994
  a Red Herring: 1986
  juxtaposition: 1981
  a story within a story: 1961
  Checkhov's gun: 1958
  a moral lesson: 1950
  absence indicating a presence: 1949
  symbolism: 1

In [18]:
output_file = 'proposed_leaf_nodes.json'
with open(output_file, 'w') as f:
    json.dump(leaf_nodes, f, indent=2)

## Step 2: Initialize Codebook Generator

Set up the generator with OpenAI API (reads from environment variables or .env file).


In [19]:
from generate_codebooks import CodebookGenerator
generator = CodebookGenerator(model="gpt-5-mini-2025-08-07")

# Load the proposed leaf nodes
leaf_nodes = generator.load_leaf_nodes(output_file)
print(f"Loaded {len(leaf_nodes)} leaf nodes")

Loaded 50 leaf nodes


## Step 3: Generate a Test Codebook

Generate a small test codebook to verify everything works.


In [20]:
# Generate a small test codebook
test_codebook = generator.generate_codebook(
    leaf_nodes[:8],  # Use first 8 leaf nodes
    size="small",
    difficulty="easy",
    use_all_formulas=False
)

print("Generated Codebook:")
print("=" * 60)
print(test_codebook)
print("=" * 60)


Generated Codebook:
[TOPIC-MYSTICAL-CREATURES]
A story is [TOPIC-MYSTICAL-CREATURES] if the story is about mystical creatures

[TOPIC-THE-ARTS]
A story is [TOPIC-THE-ARTS] if the story is about the arts

[TOPIC-GIANT-CREATURES]
A story is [TOPIC-GIANT-CREATURES] if the story is about giant creatures

[TOPIC-BYGONE-ERAS]
A story is [TOPIC-BYGONE-ERAS] if the story is about bygone eras

[TOPIC-CULTURAL-TRADITIONS]
A story is [TOPIC-CULTURAL-TRADITIONS] if the story is about cultural traditions

[TOPIC-SEASONAL-CHANGES]
A story is [TOPIC-SEASONAL-CHANGES] if the story is about seasonal changes

[FANTASTICAL-HERITAGE]
A story is [FANTASTICAL-HERITAGE] if either of the following is true:
- The story is [TOPIC-MYSTICAL-CREATURES]
- Both of the following are true:
  - The story is [TOPIC-CULTURAL-TRADITIONS]
  - The story is [TOPIC-BYGONE-ERAS]
- Both of the following are true:
  - The story is [TOPIC-THE-ARTS]
  - The story is [TOPIC-SEASONAL-CHANGES]


## Step 4: Generate Obfuscated Version

Create an obfuscated version where nodes are renamed to uninformative names (attr-1, attr-2, etc.).


In [21]:
# Generate obfuscated version
obfuscated_codebook = generator.obfuscate_codebook(test_codebook)

print("Obfuscated Codebook:")
print("=" * 60)
print(obfuscated_codebook)
print("=" * 60)


Obfuscated Codebook:
[attr-5]
A story is [attr-5] if the story is about mystical creatures

[attr-7]
A story is [attr-7] if the story is about the arts

[attr-4]
A story is [attr-4] if the story is about giant creatures

[attr-2]
A story is [attr-2] if the story is about bygone eras

[attr-3]
A story is [attr-3] if the story is about cultural traditions

[attr-6]
A story is [attr-6] if the story is about seasonal changes

[attr-1]
A story is [attr-1] if either of the following is true:
- The story is [attr-5]
- Both of the following are true:
  - The story is [attr-3]
  - The story is [attr-2]
- Both of the following are true:
  - The story is [attr-7]
  - The story is [attr-6]


## Step 5: Generate Multiple Codebooks

Generate codebooks of different sizes and difficulties. This will create:
- 20 small codebooks (3-5 nodes)
- 20 medium codebooks (6-10 nodes)  
- 10 large codebooks (11-20 nodes)

Each codebook will also have an obfuscated version saved as `-obfc.txt`.


In [24]:
output_dir = "test-1"

generator.generate_all_codebooks(
    output_dir=output_dir,
    small_count=1,
    medium_count=1,
    large_count=1
)

Small codebooks:   0%|          | 0/1 [00:00<?, ?it/s]

Small codebooks: 100%|██████████| 1/1 [00:52<00:00, 52.33s/it]


Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-001-small-easy-allf.txt
Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-001-small-easy-allf-obfc.txt


Medium codebooks: 100%|██████████| 1/1 [00:41<00:00, 41.35s/it]


Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-002-medium-medium-allf.txt
Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-002-medium-medium-allf-obfc.txt


Large codebooks: 100%|██████████| 1/1 [01:04<00:00, 64.59s/it]

Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-003-large-hard-allf.txt
Saved: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1/cb-003-large-hard-allf-obfc.txt

✓ Generated 3 codebooks (and obfuscated versions)
  Output directory: /home/jjb/msc/axiom-guided-structured-reasoning/codebooks/generator/test-1


## Step 6: Generate Individual Codebooks

Generate specific codebooks with custom parameters.


In [25]:
medium_codebook = generator.generate_codebook(
    leaf_nodes,
    size="medium",
    difficulty="easy",
    use_all_formulas=True
)

print("Medium Codebook with All Formulas:")
print("=" * 60)
print(medium_codebook)
print("=" * 60)

# Save it
generator.save_codebook(medium_codebook, "example-medium.txt", output_dir=output_dir)

# Generate and save obfuscated version
obfuscated_medium = generator.obfuscate_codebook(medium_codebook)
generator.save_codebook(obfuscated_medium, "example-medium-obfc.txt", output_dir=output_dir)


Medium Codebook with All Formulas:
[STYLE-LIGHTHEARTED]
A story is STYLE-LIGHTHEARTED if the story is lighthearted.

[THEME-HARDSHIP]
A story is THEME-HARDSHIP if the story has the theme Hardship.

[TOPIC-THE-ARTS]
A story is TOPIC-THE-ARTS if the story is about the arts.

[THEME-GROWTH]
A story is THEME-GROWTH if the story has the theme Growth.

[USES-WH-QUESTIONS]
A story is USES-WH-QUESTIONS if the story uses wh-questions.

[STARTS-WITH-ADVERB]
A story is STARTS-WITH-ADVERB if the story starts with a adverb.

[STYLE-CLASSIC]
A story is STYLE-CLASSIC if the story is classic.

[MOOD-XOR]
A story is MOOD-XOR if exactly one of the following is true:
- The story is [STYLE-LIGHTHEARTED]
- The story is [STYLE-CLASSIC]

[SERENE]
A story is SERENE if both of the following are true:
- The story is [STYLE-LIGHTHEARTED]
- The story is not [THEME-HARDSHIP]

[QUEST-OR-GROW]
A story is QUEST-OR-GROW if either of the following are true:
- The story is [USES-WH-QUESTIONS]
- The story is [THEME-GROWT

## Step 7: Verify Generated Codebooks

Parse a generated codebook to verify it's valid.


In [28]:
test_codebook = medium_codebook

from parser import CodebookParser

parser = CodebookParser()

test_file = Path(output_dir) / "test-codebook.txt"
test_file.parent.mkdir(exist_ok=True)
with open(test_file, 'w') as f:
    f.write(test_codebook)

try:
    graph = parser.parse_codebook(str(test_file))
    print(f"✓ Successfully parsed codebook!")
    print(f"  Nodes: {len(graph.nodes)}")
    print(f"  Edges: {len(graph.edges)}")
    print(f"\nNodes:")
    for node in graph.nodes:
        formula_type = type(node.formula).__name__ if node.formula else "None"
        print(f"  - {node.id}: {formula_type}")
except Exception as e:
    print(f"✗ Error parsing codebook: {e}")


✗ Error parsing codebook: In formula requires at least 2 arguments, got 1
